In [9]:
# For Submission
import os
import numpy as np
import pandas as pd
from PIL import Image
import random

def rle_encode(mask):
    """
    Convert mask to RLE (Run Length Encoding).
    """
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def trimap_to_rle_csv(folder_path, output_csv_path):
    """
    Convert all trimap images in a folder to RLE-encoded CSV.
    
    Parameters:
        folder_path (str): Path to the folder containing trimap images.
        output_csv_path (str): Path to save the output CSV file.
    """
    rle_data = []
    
    # Get a sorted list of files based on the numeric value in their names
    files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]
    files.sort(key=lambda x: int(os.path.splitext(x)[0]))  # Sort by numeric part of the filename

    # Generate IDs from 1_1 to 2000_3
    ids = [f"{i}_{j}" for i in range(1, 2001) for j in range(1, 4)]
    
    # Create a dictionary to track mask values
    id_to_mask = {id_: '' for id_ in ids}  # Initialize all IDs with empty masks

    # Iterate through all sorted files in the folder
    for filename in files:
        trimap_path = os.path.join(folder_path, filename)
        trimap = Image.open(trimap_path).convert("L")
        trimap_np = np.array(trimap)

        # RLE encode each unique class in the trimap
        for label in np.unique(trimap_np):
            mask = (trimap_np == label).astype(np.uint8)
            rle = rle_encode(mask)
            # Assign RLE to the appropriate ID if it exists
            id_key = f"{os.path.splitext(filename)[0]}_{label}"
            if id_key in id_to_mask:
                id_to_mask[id_key] = rle
    
    # Prepare data for DataFrame, ensuring we have 6000 entries
    for id_ in ids:
        rle_data.append({'id': id_, 'mask': id_to_mask[id_]})

    # Check if we have exactly 6000 rows
    if len(rle_data) != 6000:
        raise ValueError(f"Expected 6000 rows, but got {len(rle_data)}")

    # Save the RLE data to a CSV file
    rle_df = pd.DataFrame(rle_data)
    rle_df = rle_df[['id', 'mask']]
    rle_df.to_csv(output_csv_path, index=False)
    print(f"RLE CSV saved to: {output_csv_path}")

# Usage
folder_path = '/Users/bohan/Desktop/ECSE_415/ECSE415/Oxford' # Replace with your trimap file path
output_csv_path = '/Users/bohan/Desktop/ECSE_415/ECSE415/output.csv'  # Specify a full file path for the output CSV
trimap_to_rle_csv(folder_path, output_csv_path)

# path = '/Users/bohan/Desktop/ECSE 415/ECSE415/Oxford'


RLE CSV saved to: /Users/bohan/Desktop/ECSE_415/ECSE415/output.csv
